# CSC 480-F25 Lab 2: Task Decomposition, Multi‑Agent Design Patterns, and Communication Protocols

# Authors:

***Kaila Aquino***

California Polytechnic State University, San Luis Obispo;

Computer Science & Software Engineering Department

# Overview

This lab focuses on:
- Breaking down complex problems into smaller tasks (task decomposition)
- Defining specialized agents with clear roles and responsibilities  
- Choosing appropriate multi-agent design patterns
- Specifying communication protocols between agents
- Implementing a three-agent system using AutoGen

NOTE: If you already know what you're going to do for your final project, you may use this lab to make progress on it. Otherwise, treat this lab as a distinct exercise.

## Learning Objectives

By the end of this lab, you will be able to:

- Decompose a complex problem into a comprehensive set of constituent tasks
- Identify and define the roles, responsibilities, and abilities of individual agents
- Select and justify an appropriate multi-agent design pattern (Manager‑Worker, Sequential Pipeline, Collaborative Team)
- Explain and apply foundational agent communication protocols at a high level (MCP, A2A)
- Start thinking about the architecture of your project (agents, tasks, design pattern, and interaction protocols)

# Part 1: Task Decomposition and Agent Design

## 1. Problem Statement & Task Decomposition

**Main Problem:** Suggests a scenic route to a specific destination given a rough time estimate

**Task Breakdown:** List the steps and sub-tasks needed to achieve the goal:
1. Gather the input from the user, the destination and desired driving time
2. Planner: 
    * Look at the map to gather the "normal route" 
    * Take note of other roads / highways to the destination 
    * Reference OpenStreetMaps to look for roads with higher speed limits, areas away from commercial buildings, near hills or the coast
    * Create a list of possible routes 
3. Implementer:
    * Fetch each route possibility
    * Compare routes to see which is the most "scenic" by scoring them 
        * Formulate a way to score: 
        * Scenic score: defined by if it is near mountains or water, away from the regular road, etc
        * From the scenic score, pick the highest scoring ones that are closest to the time estimate 
            * rank by scenic score, then rank by time estimate 
    * Returns top 3 most scenic routes
4. Critic: 
    * Check if routes meets the scenic and time criteria 
    * If not scenic enough or not close to time criteria, send back to implementer

## 2. Agent Definition

Define your primary agents with clear roles and responsibilities:

### Agent 1: Planner
- **Role:** Generate possible routes given the input 
- **Responsibilities:** 
    * Find the default route to the location
    * Determine how time criteria compares to default eta to figure out how much time to add / subtract if possible 
    * Look at map characteristics, speed limit, water, hills, buildings 
    * Generate scenic routes that include hills, water, faster speed limits, etc
- **Inputs:** User inputs 
    * Current location 
    * Destination
    * Time
- **Outputs:** A list of possible routes
    * List of routes must be close to the time criteria and be different from default route
- **Success Criteria:** At least one alternative route that fits the time constraint

### Agent 2: Implementer 
- **Role:** Selects top 3 routes
- **Responsibilities:** 
    * Fetch the list of routes provided by the planner 
    * Look through the qualities of each route and score each route by scenery
    * Choose 3 routes with the highest scenic score and closest to time constraint
- **Inputs:** A list of routes from the planner
- **Outputs:** 3 scenic routes
- **Success Criteria:** 3 detailed routes that follows time constraints and is feasible by the user. 

### Agent 3: Critic/Integrator
- **Role:** Evaluates the routes and selects the best fit given the criteria. 
- **Responsibilities:** 
    * Analyze each route given by the Implementer
    * Score each route to determine which one has the best balance of scenic quality and time 
    * Ensure that the chosen route is different from the default route 
- **Inputs:** List of 3 routes
- **Outputs:** One final route 
- **Success Criteria:** The final route is different from the default route and is close to time constraint 

## 3. Design Pattern Selection

**Chosen Pattern:** [Manager-Worker / **Sequential Pipeline** / Collaborative Team]

**Justification:** The overall task can be broken down into smaller steps. Each step must be completed before moving on to a new task. Sequential pipeline is the best pattern for my workflow because the Planner must finish generating routes before the Implementer can choose top 3 routes and the Critic has to wait for both of them before evaluating the routes and choosing the final route. There are no overlapping tasks between agents and each task must be completed in order. If any revisions must be done, the workflow can reset at the planning step or the Implementer can re-do its job. 

## 4. Communication Design

### Model Context Protocol (MCP)
**Shared Context & Tools:** 
    * User input: Current location, destination and time criteria 
    * Map data: Roads, highways, elevation, speed limit, bodies of water, buildings, OpenStreetMaps
    * Route canidiates generated by the Planner
      
- Access rules: 
    * All users have access to the user input and map data 
    * Only the planner can create route candidates
    * Only the implementer can score the routes based on scenic score and time feasibility
    * The critic can comprehensively score the top 3 routes to choose the best route to return to the user

### Agent-to-Agent (A2A) Interactions

#### Interaction 1: Planner → Implementer
- **Purpose:** Give the implenter a list of routes to choose from 
- **Key Fields:** route id, distance, time, route path 
- **Message Format:** JSON 
#### Interaction 2: Implementer → Critic 
- **Purpose:** Provide top 3 selected routes 
- **Key Fields:** route id, distance, time, route path, scenic score
- **Message Format:** JSON


## 5. Interaction Diagram

[Create a flowchart or sequence diagram showing agents, your chosen pattern, and MCP/A2A interactions. You can use text-based diagrams, draw by hand and insert an image, or use diagramming tools.]

```
Example text-based flow:
User      →    Planner    →    Implementer    →    Critic    →    User

  ↓              ↓                 ↓                 ↓

  MCP          A2A               A2A              Final
  Tool         Msg1              Msg2            Output
User Input


```

# Part 2: Three-Agent AutoGen Implementation

## Environment Setup

Install required packages and set up Azure OpenAI configuration.

In [1]:
pip install python-dotenv


[notice] A new release of pip is available: 24.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 2.6 MB/s eta 0:00:00
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.0 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.0 MB/s eta 0:00:00
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.2 MB/s eta 0:00:

In [4]:
import os
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


from dotenv import load_dotenv
load_dotenv()

True

In [6]:
azure_deployment = os.getenv("AZURE_DEPLOYMENT")
api_version = os.getenv("AZURE_API_VERSION")
azure_endpoint = os.getenv("AZURE_ENDPOINT")
api_key = os.getenv("AZURE_SUBSCRIPTION_KEY")

In [11]:
# This is the same as in L1
# azure_deployment = "your-deployment-name"
# api_version = "2024-12-01-preview"  # If you're using gpt-5-mini as demonstrated

# # e.g. https://ccoha-mfoynknp-eastus2.cognitiveservices.azure.com/
# azure_endpoint = "your-azure-endpoint"


## An Example Three-Agent System Architecture

Based on the suggested roles from the overview:
- **Planner**: Decomposes the user goal into a concise, ordered plan
- **Implementer**: Executes the steps from the plan
- **Critic/Integrator**: Reviews output and suggests improvements or approves completion

In [7]:
async def setup_three_agent_system():
    """Set up the three-agent system with Planner, Implementer, and Critic roles"""
    
    # Create the Azure OpenAI client
    client = AzureOpenAIChatCompletionClient(
        azure_deployment=azure_deployment,
        model="gpt-5-mini",
        api_version=api_version,
        azure_endpoint=azure_endpoint,
        api_key=os.getenv("AZURE_SUBSCRIPTION_KEY"),
    )
    
    
    # Define the three agents with specific system prompts
    planner = AssistantAgent(
        name="Planner",
        model_client=client,
        system_message="""You are a task decomposition specialist. Your role is to:
        1. Break down complex user goals into clear, numbered steps
        2. Define specific inputs/outputs for each step  
        3. Provide testable acceptance criteria for each step
        4. Present plans in a structured, actionable format
        
        Always end your response with 'PLAN_COMPLETE' when finished."""
    )
    
    implementer = AssistantAgent(
        name="Implementer", 
        model_client=client,
        system_message="""You are an implementation specialist. Your role is to:
        1. Follow the Planner's steps methodically
        2. Address each step with concrete outputs/artifacts
        3. Provide detailed solutions that meet the specified criteria
        4. Ask for clarification if any step is unclear
        
        Always end your response with 'IMPLEMENTATION_COMPLETE' when finished."""
    )
    
    critic = AssistantAgent(
        name="Critic",
        model_client=client, 
        system_message="""You are a quality assurance specialist. Your role is to:
        1. Review the Implementer's work against the Planner's criteria
        2. Identify gaps, errors, or missing elements
        3. Suggest specific improvements if needed
        4. Only approve with 'APPROVED' if all criteria are fully met
        
        Be thorough but constructive in your feedback."""
    )
    
    return planner, implementer, critic

In [8]:
async def run_three_agent_workflow(task_description):
    """Run the three-agent workflow for a given task"""

    # Set up the agents
    planner, implementer, critic = await setup_three_agent_system()

    # Create termination condition - look for "APPROVED" in messages
    # See https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TerminationCondition
    termination = TextMentionTermination("APPROVED")

    # Create a group chat with the three agents
    # The RoundRobinGroupChat will manage turn-taking between agents
    # See https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat
    team = RoundRobinGroupChat([planner, implementer, critic], termination_condition=termination)

    # Run the workflow
    print("Starting three-agent workflow...")
    print("=" * 60)

    # Start the conversation
    result = await Console(
        team.run_stream(task=task_description)
    )

    return result

## Example Task 1: Analytical Task

Test the three-agent system with a small example analytical task.

In [8]:
# Test Task 1: Compare sorting algorithms
analytical_task = """
Compare the pros and cons of Quick Sort vs Merge Sort algorithms. 
Provide a structured analysis that includes:
- Time and space complexity for both
- Best/worst case scenarios  
- Practical use cases for each
- A final recommendation with justification
"""

# Uncomment to run when ready:
await run_three_agent_workflow(analytical_task)

Starting three-agent workflow...
---------- TextMessage (user) ----------

Compare the pros and cons of Quick Sort vs Merge Sort algorithms. 
Provide a structured analysis that includes:
- Time and space complexity for both
- Best/worst case scenarios  
- Practical use cases for each
- A final recommendation with justification

---------- TextMessage (Planner) ----------
1) Step 1 — Provide concise algorithm overviews
- Input: Names "Quick Sort" and "Merge Sort"
- Output: Short, clear descriptions of how each algorithm works (key steps)
- Content:
  - Quick Sort: A divide-and-conquer, in-place partitioning sort. Choose a pivot, partition the array into elements < pivot and > pivot, then recursively sort partitions. Typical implementations use Lomuto or Hoare partition schemes; randomized pivot or median-of-three can reduce worst-case risk.
  - Merge Sort: A divide-and-conquer stable sort. Recursively split the array (or list) in half, sort each half, then merge the two sorted halves in

TaskResult(messages=[TextMessage(id='cb944977-2ef3-449d-ac5a-662bfe1b985f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 22, 16, 28, 86601, tzinfo=datetime.timezone.utc), content='\nCompare the pros and cons of Quick Sort vs Merge Sort algorithms. \nProvide a structured analysis that includes:\n- Time and space complexity for both\n- Best/worst case scenarios  \n- Practical use cases for each\n- A final recommendation with justification\n', type='TextMessage'), TextMessage(id='566c5abc-fa55-4523-afe1-c3cf5096a5fd', source='Planner', models_usage=RequestUsage(prompt_tokens=141, completion_tokens=2890), metadata={}, created_at=datetime.datetime(2025, 10, 2, 22, 17, 0, 460344, tzinfo=datetime.timezone.utc), content='1) Step 1 — Provide concise algorithm overviews\n- Input: Names "Quick Sort" and "Merge Sort"\n- Output: Short, clear descriptions of how each algorithm works (key steps)\n- Content:\n  - Quick Sort: A divide-and-conquer, in-place pa

## Example Task 2: Creative Task

Test the three-agent system with a creative writing task.

In [21]:
# Test Task 2: Creative writing with constraints  
creative_task = """
Write a 150-word summary of the benefits of renewable energy that:
- Uses exactly 3 specific statistics or data points
- Includes at least 2 types of renewable energy sources
- Has a compelling call-to-action in the final sentence
- Maintains an optimistic but factual tone throughout
"""

# Uncomment to run when ready:
# await run_three_agent_workflow(creative_task)

## Your Task

Write your own task for the planner-implementer-critic system

In [12]:
your_task = """
Find a the most scenic driving route from California Polytechnic State University San Luis Obispo to Pismo Beach, California that will take about 1 hour.
The route must be 
- different from the default route from Cal Poly Grand Ave to Pismo Beach
- scenic, drive must some of the following, water, hills, high elevation, 50-70 speed limit, away from commercial buildings
- return one route by sharing what roads I should take to get to Pismo Beach

The final route should be detailed 
- instructions on where to turn and which exit to take leaving the starting point
- specific road names where turns should be made 
- the drive time
- amount of miles driven 

When deciding on a route, refer to Google Maps and Open Street Maps for accurrate directions and to assist with scenic score. 
"""
# Uncomment to run when ready:
await run_three_agent_workflow(your_task)

Starting three-agent workflow...
---------- TextMessage (user) ----------

Find a the most scenic driving route from California Polytechnic State University San Luis Obispo to Pismo Beach, California that will take about 1 hour.
The route must be 
- different from the default route from Cal Poly Grand Ave to Pismo Beach
- scenic, drive must some of the following, water, hills, high elevation, 50-70 speed limit, away from commercial buildings
- return one route by sharing what roads I should take to get to Pismo Beach

The final route should be detailed 
- instructions on where to turn and which exit to take leaving the starting point
- specific road names where turns should be made 
- the drive time
- amount of miles driven 

When deciding on a route, refer to Google Maps and Open Street Maps for accurrate directions and to assist with scenic score. 

---------- TextMessage (Planner) ----------
I’ll break this into clear, testable steps, then give a ready-to-drive scenic route (differe

TaskResult(messages=[TextMessage(id='305f9470-f1c6-4e10-b21d-3693a3ac616c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 3, 3, 54, 18, 386502, tzinfo=datetime.timezone.utc), content='\nFind a the most scenic driving route from California Polytechnic State University San Luis Obispo to Pismo Beach, California that will take about 1 hour.\nThe route must be \n- different from the default route from Cal Poly Grand Ave to Pismo Beach\n- scenic, drive must some of the following, water, hills, high elevation, 50-70 speed limit, away from commercial buildings\n- return one route by sharing what roads I should take to get to Pismo Beach\n\nThe final route should be detailed \n- instructions on where to turn and which exit to take leaving the starting point\n- specific road names where turns should be made \n- the drive time\n- amount of miles driven \n\nWhen deciding on a route, refer to Google Maps and Open Street Maps for accurrate directions and to a

NOTE: You'll notice that the output might contain duplicated text, that is because `Console` prints when a text message is sent and received.

## Reflection & Analysis

### What worked well?
[Reflect on where the three-agent pattern performed effectively]
The three agent pattern worked well in breaking up the steps and working in a sequential work flow. The critic was effectively able to evaluate a route and request for another attempt. 
### What struggled?
The models did not have a clear vision of the map. The directions given were not accurate to the map. I tried to be more specific with accuracy but it did not help. 
### Potential improvements:
I could be more detailed with how I expect the results to be. I should have asked the model to refer to Google Maps or Open Street Maps for the road names and routes to ensure accuracy. 

Also, the model would ask me questions but I have no way to answer them so I could not refine their answer. 

### Tool integration:
[Optional: Discuss how adding tools (like a calculator function) might improve the system]

# Optional Extensions

## Adding Tools (Optional Challenge)

Consider adding a simple tool function that agents can use, such as a calculator or text analyzer.

In [ ]:
# Example tool function (uncomment and modify as needed)
# def calculator(expression: str) -> float:
#     """Simple calculator tool for basic mathematical operations"""
#     try:
#         # WARNING: eval() should not be used in production - use a proper math parser
#         result = eval(expression)
#         return result
#     except Exception as e:
#         return f"Error: {str(e)}"

# To integrate tools with AutoGen agents, you would typically:
# 1. Define tool schemas following MCP-like patterns
# 2. Register tools with specific agents  
# 3. Update system prompts to mention available tools
# 4. Handle tool calls in the conversation flow

# Summary and Next Steps

## Key Takeaways

- **Task Decomposition**: [Your insights about breaking down complex problems]
- **Agent Design**: [What you learned about defining agent roles and responsibilities]  
- **Communication Protocols**: [Understanding of MCP and A2A interactions]

## References

- [AutoGen Documentation](https://microsoft.github.io/autogen/stable/index.html)
- [Model Context Protocol](https://modelcontextprotocol.io/docs/getting-started/intro)
- [Agent-to-Agent Protocol](https://a2a-protocol.org/latest/)
- L2_overview.md (Lab requirements and detailed explanations)